In [1]:
#import libraries
import os
import json
import math
import csv
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from shapely.geometry import Polygon as ShapelyPolygon
from matplotlib.collections import PatchCollection
import pandas as pd
import numpy as np
from matplotlib.patches import Rectangle
from tabulate import tabulate

In [2]:
def read_file(path):
    with open(path, 'r') as file:
        # Read the content of the file
        file_content = file.read()
    return file_content

In [3]:
def parse_text_blocks(content):
    lines = content.split('\n')
    data = []
    id_counter = 0
    for line in lines:
        if line.strip() == '':
            continue
        values = line.split()
        if len(values) < 5:
            continue
        x0, y0, x1, y1 = map(int, values[:4])
        text = ' '.join(values[4:-1])  # Exclude the last value which represents text_role
        text_role = values[-1]
        polygon = {
            'x0': x0,
            'x1': x1,
            'x2': x1,
            'x3': x0,
            'y0': y0,
            'y1': y0,
            'y2': y1,
            'y3': y1
        }
        data.append({'id': id_counter, 'polygon': polygon, 'text': text, 'text_role': text_role})
        id_counter += 1
    return data

In [4]:
def parse_tick_data(content):
    tick_data = {'x-axis': [], 'y-axis': []}
    lines = content.split('\n')
    id_counter = 0
    for line in lines:
        if line.strip() == '':
            continue
        values = line.split()
#         print(f"values = {values}")
        x0, y0, x1, y1, text_role, axis_type, x, y = values
        x0 = int(x0)
        y0 = int(y0)
        x1 = int(x1)
        y1 = int(y1)
#         x0, y0, x1, y1 = map(int, x0)
        x, y = map(int, (x, y))
        tick_point = {'x': x, 'y': y}
        tick_info = {'id': id_counter, 'tick_pt': tick_point}
        if 'x_axis' in axis_type:
            tick_data['x-axis'].append(tick_info)
        else:
            tick_data['y-axis'].append(tick_info)
        id_counter += 1
    return tick_data

In [5]:
def calculate_intersection_over_area(detected_bar, ground_truth_bar):
    # Calculate coordinates of intersection rectangle
    x_left = max(detected_bar['x0'], ground_truth_bar['x0'])
    y_bottom = max(detected_bar['y0'], ground_truth_bar['y0'])
    x_right = min(detected_bar['x0'] + detected_bar['width'], ground_truth_bar['x0'] + ground_truth_bar['width'])
    y_top = min(detected_bar['y0'] + detected_bar['height'], ground_truth_bar['y0'] + ground_truth_bar['height'])
    
    # Calculate intersection area
    intersection_area = max(0, x_right - x_left) * max(0, y_top - y_bottom)
    
    # Calculate IoA
    detected_area = detected_bar['width'] * detected_bar['height']
    ground_truth_area = ground_truth_bar['width'] * ground_truth_bar['height']
    IoA = intersection_area / (detected_area + ground_truth_area - intersection_area)
    
    return IoA

def map_bars(detected_bars, ground_truth_bars):
    mapped_bars = []
    gt_bars = {}
    dt_bars = {}
    idd = 1
    for gt_bar in ground_truth_bars:
        max_IoA = 0
        mapped_bar = None
        for dt_bar in detected_bars:
#             print(f"dt_bar = {dt_bar}")
#             print(f"gt_bar = {gt_bar}")
            IoA = calculate_intersection_over_area(dt_bar, gt_bar)
#             print(IoA)
            if IoA > max_IoA:
                max_IoA = IoA
                mapped_bar = dt_bar
        if mapped_bar is not None:
            gt_bars[idd] = gt_bar
            dt_bars[idd] = mapped_bar
            idd += 1
            mapped_bars.append((gt_bar, mapped_bar))
            detected_bars.remove(mapped_bar)
    
    return gt_bars,dt_bars

In [6]:
def parse_bar_data(content):
    bars = []
    lines = content.split('\n')
    for line in lines:
        if line.strip() == '':
            continue
        values = line.split()
        x0, y0, x1, y1 = map(int, values)
        height = abs(y1 - y0)
        width = abs(x1 - x0)
        bars.append({'height': height, 'width': width, 'x0': x0, 'y0': y0})
    return {'bars': bars}

In [7]:
def polygon_to_key(polygon):
    # Convert the polygon to a hashable tuple representation
    return tuple(sorted(polygon.items()))


def find_polygon_center(vertices):
    num_vertices = len(vertices)

    if num_vertices < 3:
        raise ValueError("A polygon must have at least 3 vertices.")

    sum_x = sum(v[0] for v in vertices)
    sum_y = sum(v[1] for v in vertices)

    center_x = sum_x / num_vertices
    center_y = sum_y / num_vertices

    return center_x, center_y

def assign_polygons_to_ticks(xaxis_ticks, yaxis_ticks, polygons):
    assigned_xaxis_polygons = []
    assigned_yaxis_polygons = []
    box_tick_mapping = []


    for polygon in polygons:
        idd = polygon['id']
        text_role = polygon['text_role']
        
#         temp =  text_roles.loc[text_roles['text'] == polygon['text'],'role'].values[0]
        if text_role != 'tick_label':
            continue
            
        polygon_vertices = [
            (polygon['polygon'][f'x{i}'], polygon['polygon'][f'y{i}']) for i in range(4)
        ]
        center = find_polygon_center(polygon_vertices)

        # Find the nearest tick label (x, y) coordinate
#         x_distances = [abs(center[0] - tick['tick_pt']['x']) for tick in xaxis_ticks]
#         y_distances = [abs(center[1] - tick['tick_pt']['y']) for tick in yaxis_ticks]
        
        x_distances = 0
        y_distances = 0
        
        x1 = center[0]
        y1 = center[1]
        
        min_xtick = []
        min_ytick = []
        
        for item in xaxis_ticks:
            x2 = item['tick_pt']['x']
            y2 = item['tick_pt']['y']
            dist = math.sqrt((x2-x1) ** 2 + (y2-y1)**2)
            
            if x_distances == 0:
                x_distances = dist
                min_xtick.append((x2,y2))
            elif x_distances > dist:
                x_distances = dist
                min_xtick[0] = ((x2,y2))
                
        
        for item in yaxis_ticks:
            x2 = item['tick_pt']['x']
            y2 = item['tick_pt']['y']
            dist = math.sqrt((x2-x1)**2 + (y2-y1)**2)
            
            if y_distances == 0:
                y_distances = dist
                min_ytick.append((x2,y2))
            elif y_distances > dist:
                y_distances =  dist
                min_ytick.append((x2,y2))
                
        if len(xaxis_ticks) == 0:
            assigned_yaxis_polygons.append(polygon)
            box_tick_mapping.append((polygon, min_ytick))
            continue
        
    
        if x_distances <= y_distances:
            assigned_xaxis_polygons.append(polygon)
            box_tick_mapping.append((polygon,min_xtick))
        else:
            assigned_yaxis_polygons.append(polygon)
            box_tick_mapping.append((polygon,min_ytick))

    return assigned_xaxis_polygons, assigned_yaxis_polygons, box_tick_mapping

In [8]:
def per_pixel_diff(yaxis_ticks,yaxis_polygons):
#     print(f"yaxis_ticks = {yaxis_ticks}")
    yaxis_polygons = sorted(yaxis_polygons, key=lambda x: x['polygon']['y0'])
    pixel_diff = abs(yaxis_ticks[0]['tick_pt']['y'] - yaxis_ticks[1]['tick_pt']['y'])
    yy1 = float(yaxis_polygons[0]['text'].replace("%", ""))
    yy2 = float(yaxis_polygons[1]['text'].replace("%",""))
#     print(f"yy1 = {yy1} yy2 = {yy2} yaxis_ticks[0] = {yaxis_ticks[0]} yaxis_ticks[1] = {yaxis_ticks[1]}")
    val_diff_bw_two_coord = abs(yy1 - yy2)
    one_pixel_val = val_diff_bw_two_coord / pixel_diff
    return one_pixel_val

In [9]:
def sort_polygons(polygons):
    sorted_polygons = sorted(polygons, key=lambda x: x['polygon']['x0'])
    return sorted_polygons

In [10]:
def sort_bars(bars):
    sorted_bars = sorted(bars['bars'], key=lambda x: x['x0'])
    return sorted_bars

In [11]:
def predict_bar_values2(bars, yaxis_ticks,yaxis_polygons):
    predicted_y_values = []
    
    sorted_bars = sorted(bars.items(), key=lambda x: x[1]['x0'])
    
#     print(yaxis_ticks)
    
    one_pixel_val = per_pixel_diff(yaxis_ticks,yaxis_polygons)
    
    for bar in sorted_bars:
#         print(bar[1])
        value = bar[1]['height'] * one_pixel_val
        predicted_y_values.append((bar,value))
    
    return predicted_y_values;

In [12]:
def find_legend_pairs(polygons):
    legend_pairs = []
    for item in polygons:
        if item['text_role'] == 'legend_label':
            legend_pairs.append(item)
    
    return legend_pairs

In [13]:
def data_extraction(predicted_y_values, polygons,legend_pairs):
    sorted_polygons = sort_polygons(polygons)
    
    data_mapping = {}
    
    for polygon in polygons:
        data_mapping[polygon['text']] = []
    
    
    for item in predicted_y_values:
        bar = item[0][1]
        min_dist = 0
        y_mapped_polygon = {}
        for polygon in polygons:
            polygon_vertices = [
            (polygon['polygon'][f'x{i}'], polygon['polygon'][f'y{i}']) for i in range(4)]
            center = find_polygon_center(polygon_vertices)
            
            x1, y1 = center[0], center[1]
            
            x2 = bar['x0'] + bar['width']/2
            y2 = (bar['y0'] + bar['height'])
            
            dist =  math.sqrt((x2-x1)**2 + (y2-y1)**2)
            
            if min_dist == 0 or min_dist > dist:
                y_mapped_polygon[item[0][0]] = polygon
                min_dist = dist
                
        
        data_mapping[y_mapped_polygon[item[0][0]]['text']].append(item)
            
    return data_mapping

In [14]:
def parse_gt_bar_data(content):
    bars = []
    lines = content.split('\n')
    for line in lines:
        if line.strip() == '':
            continue
        values = line.split()
        x0, y0, x1, y1 = map(int, values[:-1])
        value = float(values[-1])
        height = abs(y1 - y0)
        width = abs(x1 - x0)
        bars.append({'height': height, 'width': width, 'x0': x0, 'y0': y0, 'value':value})
    return {'bars': bars}

In [15]:
image_path = "./images/"
text_detect_role_path = "./text_detect_role_classify/"
bar_plot_path = './Bar_plot_coordinates/'
axis_analysis_path = './Axis_Analysis2/'
gt_coordinates_path = "./GT_coordinate_value2/"


In [20]:
def dataTable(filename, text_result, data_mapping):
    legend_pairs = find_legend_pairs(text_result)

    num_of_legends = len(legend_pairs)

    headers = []


    headers.append("")

    for item in legend_pairs:
        headers.append(item['text'])


    if num_of_legends == 0:
        headers.append("value")

    table = []

    for item in data_mapping.items():
        temp = []
#         print(f"item = {item}")
        temp.append(item[0])
        cnt = 0
#         print(temp)
#         print(item[1])
#         print(len(item[1]))
        if len(item[1]) > num_of_legends and num_of_legends != 0:
            print(f"dataTable = {filename}")
            break
        for value in item[1]:
#             print(f"value = {value}")
            temp.append(value[1])
            cnt += 1
   
        while cnt < num_of_legends:
            temp.append(0)
            cnt += 1

        table.append(temp)

    
    csv_filename = filename.rsplit('.', 1)[0] + '.csv'

    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        writer.writerow(headers)

        writer.writerows(table)

            

In [21]:
for filename in os.listdir(text_detect_role_path):
#     print(filename)
    image_name = image_path + filename
    text_detect_filename = text_detect_role_path + filename
    bar_plot_filename = bar_plot_path + filename
    axis_analysis_filename = axis_analysis_path + filename
    gt_coordinates_filename = gt_coordinates_path + filename
    
    # text detect role
    file_content = read_file(text_detect_filename)
    result = parse_text_blocks(file_content)

    #axis analysis
    axis_file_content = read_file(axis_analysis_filename)
    axis_result = parse_tick_data(axis_file_content)

    #bar analysis
    bar_file_content = read_file(bar_plot_filename)
    bar_results = parse_bar_data(bar_file_content)

    #gt_coordinates
    gt_coordinates = read_file(gt_coordinates_filename)
    gt_bar_results = parse_gt_bar_data(gt_coordinates)
    
    try:
        gt_bars, dt_bars = map_bars(bar_results['bars'], gt_bar_results['bars'])

        xaxis_ticks = axis_result['x-axis']
        yaxis_ticks = axis_result['y-axis']
        xaxis_polygons, yaxis_polygons, box_tick_mapping = assign_polygons_to_ticks(xaxis_ticks, yaxis_ticks, result)

        temp_xaxis_polygon = xaxis_polygons
        for polygon in temp_xaxis_polygon:
            if polygon['text'] == '0':
                xaxis_polygons.remove(polygon)


        predicted_y_values = predict_bar_values2(dt_bars, yaxis_ticks,yaxis_polygons)

        legend_pairs = find_legend_pairs(result)
        data_mapping = data_extraction(predicted_y_values, xaxis_polygons,legend_pairs)
        
        dataTable(filename, result, data_mapping )

    except:
        print(f"{filename}")

#     print(data_mapping)
    

dataTable = PMC5769657___g3.txt
PMC5732439___1_HTML.txt
dataTable = PMC3605454___g009.txt
dataTable = PMC3594293___g008.txt
dataTable = PMC5799093___Figv_HTML.txt
PMC5657859___2.txt
PMC3389018___g003.txt
dataTable = PMC3522381___g002.txt
dataTable = PMC5826268___7-figsupp1.txt
PMC3250409___g002.txt
PMC5768164___g005.txt
dataTable = PMC2737149___10.txt
dataTable = PMC4223572___2_HTML.txt
dataTable = PMC5799305___3_HTML.txt
dataTable = PMC5736661___5_HTML.txt
